In [1]:
import pandas as pd

In [3]:
patient_data = pd.read_excel('/content/Patient sale.xlsx')     # load patient data
patient_data.head()

,patient_id,drug_id,quantity_sold
0,15189111684,8956233,2
1,15189111684,8994447,1
2,15189111684,38125989,3
3,15189111684,22101453,3
4,15189111684,40659399,3


In [5]:
drug_data = pd.read_csv('/content/drugs data.csv')       # load drug data
drug_data.head()

,drug-id,type,composition
0,99,ethical,NaN
1,297,ethical,Testosterone(40mg)
2,495,ethical,Pregabalin(75mg)
3,693,ethical,"Amoxicillin(250mg),Cloxacillin(250mg),Lactobac..."
4,891,ethical,"Diclofenac(50mg),Thiocolchicoside(8mg)"


In [23]:
df = pd.merge(patient_data, drug_data, left_on ='drug_id', right_on='drug-id',how= 'inner')   # inner join drug data and patient data
df.head()

,patient_id,drug_id,quantity_sold,drug-id,type,composition
0,15189111684,8956233,2,8956233,generic,"Folic Acid(1.5mg),Iron(100mg)"
1,1204504290,8956233,1,8956233,generic,"Folic Acid(1.5mg),Iron(100mg)"
2,16709990283,8956233,2,8956233,generic,"Folic Acid(1.5mg),Iron(100mg)"
3,3393055548,8956233,3,8956233,generic,"Folic Acid(1.5mg),Iron(100mg)"
4,1314708975,8956233,3,8956233,generic,"Folic Acid(1.5mg),Iron(100mg)"


In [24]:
print(f'Total medical records: {len(df)}')

Total medical records: 18465


In [25]:
unique_patients = df['patient_id'].unique()
print(f'Total number of patients: {len(unique_patients)}')

Total number of patients: 3533


In [26]:
df['type'].value_counts()

ethical                  8718
generic                  8251
surgical                  566
ayurvedic                 408
general                   170
otc                       103
discontinued-products      77
category-4                 75
baby-product               59
baby-food                  35
high-value-ethical          2
glucose-test-kit            1
Name: type, dtype: int64

In [29]:
df1 = df.loc[(df['type']=='generic') | (df['type']=='ethical')]     # filter medical type ethical and generic

In [30]:
df1.head()

,patient_id,drug_id,quantity_sold,drug-id,type,composition
0,15189111684,8956233,2,8956233,generic,"Folic Acid(1.5mg),Iron(100mg)"
1,1204504290,8956233,1,8956233,generic,"Folic Acid(1.5mg),Iron(100mg)"
2,16709990283,8956233,2,8956233,generic,"Folic Acid(1.5mg),Iron(100mg)"
3,3393055548,8956233,3,8956233,generic,"Folic Acid(1.5mg),Iron(100mg)"
4,1314708975,8956233,3,8956233,generic,"Folic Acid(1.5mg),Iron(100mg)"


In [31]:
df1['type'].value_counts()

ethical    8718
generic    8251
Name: type, dtype: int64

In [65]:
type_seperation = pd.get_dummies(df1['type'])    # one hot encoding `type` column
type_seperation

,ethical,generic
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
18460,1,0
18461,1,0
18462,1,0
18463,1,0


In [66]:
df2 = pd.concat([df1,type_seperation],axis=1)      # join type_seperation into df2
df2 = df2.drop(['drug-id','type'], axis=1)         #  drop `drug-id` column as it is duplicated and `type` column

In [67]:
df2.head()

,patient_id,drug_id,quantity_sold,composition,ethical,generic
0,15189111684,8956233,2,"Folic Acid(1.5mg),Iron(100mg)",0,1
1,1204504290,8956233,1,"Folic Acid(1.5mg),Iron(100mg)",0,1
2,16709990283,8956233,2,"Folic Acid(1.5mg),Iron(100mg)",0,1
3,3393055548,8956233,3,"Folic Acid(1.5mg),Iron(100mg)",0,1
4,1314708975,8956233,3,"Folic Acid(1.5mg),Iron(100mg)",0,1


In [68]:
df3 = df2.groupby('patient_id')['ethical','generic'].sum().reset_index()   # groupby patient id
df3.head()

<ipython-input-68-43dbeafe5fa0>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df3 = df2.groupby('patient_id')['ethical','generic'].sum().reset_index()   # groupby patient id


,patient_id,ethical,generic
0,4328667,1,1
1,15558426,1,1
2,17203779,1,3
3,18711270,12,1
4,24207768,2,0


In [70]:
df3['ratio'] = df3['generic']/df3['ethical']     # get ratio of generic vs ethical

In [71]:
df3.head()

,patient_id,ethical,generic,ratio
0,4328667,1,1,1.000000
1,15558426,1,1,1.000000
2,17203779,1,3,3.000000
3,18711270,12,1,0.083333
4,24207768,2,0,0.000000


In [72]:
df4 = df3.loc[df3['ratio']>1].head()   # filtering rows where the ration is above 1

In [73]:
df4.head()

,patient_id,ethical,generic,ratio
2,17203779,1,3,3.0
5,37591371,0,2,inf
6,39638322,2,18,9.0
7,44610345,0,2,inf
8,59613327,2,3,1.5


In [63]:
df5 = pd.merge(df4,df2, right_on='patient_id', left_on='patient_id', how='inner')   # merging with df2 to get composition

In [64]:
df5.head()

,patient_id,ethical_x,generic_x,ratio,drug_id,quantity_sold,composition,ethical_y,generic_y
0,17203779,1,3,3.0,7465293,6,Aspirin(75mg),1,0
1,17203779,1,3,3.0,15739713,1,Atorvastatin(40mg),0,1
2,17203779,1,3,3.0,39528621,6,Metoprolol(12.5mg),0,1
3,17203779,1,3,3.0,3101769,4,"Aspirin(75mg),Clopidogrel(75mg)",0,1
4,37591371,0,2,inf,38168361,1,"Aceclofenac(100mg),Paracetamol(325mg),Serratio...",0,1


In [77]:
composition_needed = df5['composition'].to_list()

In [79]:
print(f'Composition of drugs whose generic to ethical ratio >1 : \n{composition_needed}')

Composition of drugs whose generic to ethical ratio >1 : 
['Aspirin(75mg)', 'Atorvastatin(40mg)', 'Metoprolol(12.5mg)', 'Aspirin(75mg),Clopidogrel(75mg)', 'Aceclofenac(100mg),Paracetamol(325mg),Serratiopeptidase(15mg)', 'Pantoprazole(40mg)', 'Vitamin E(400mg)', 'Amlodipine(5mg),Telmisartan(40mg)', 'Telmisartan(40mg)', 'Pantoprazole(40mg)', 'Metoprolol(25mg)', 'Calcium(500mg),Vitamin D3(250iu)', 'Atorvastatin(20mg)', 'Atorvastatin(20mg)', 'Vitamin B1(1mg),Vitamin B2(1mg),Vitamin B3(15mg),Vitamin B6(0.5mg),Vitamin B12(1mcg)', 'Aspirin(150mg),Clopidogrel(75mg)', 'Teneligliptin(20mg)', 'Metoprolol(50mg)', 'Thiocolchicoside(4mg)', 'Glimepiride(1mg),Metformin(500mg)', 'Gliclazide(80mg),Metformin(500mg)', 'Hydroquinone Topical(2%w/w),Mometasone Topical(0.1%w/w),Tretinoin Topical(0.025%w/w)', 'Diclofenac Topical(1%w/w),Linseed Oil Topical(3%w/w),Menthol Topical(5%w/w),Methyl Salicylate Topical(10%w/w)', 'Oxcarbazepine(300mg)', 'Betahistine(48mg)', 'Baclofen(10mg)', 'Oxymetazoline Topical(0.05%